In [ ]:
!pip install xgboost==1.6.1
!pip install scikit-optimize
!pip install pygad

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

choose between xgbr, dtr and ann (1.dtr ,2.xgbr, 3.ann)

In [ ]:
#from xgboost import XGBRegressor
#new_model=XGBRegressor()
#new_model.load_model('/content/drive/MyDrive/Colab Notebooks/xgbr_avggc.h5')

#pickle for rfr
import pickle
with open('/content/drive/MyDrive/Colab Notebooks/rfr_avggc.pkl' , rb') as f:
    new_model = pickle.load(f)

#from keras import models
# Recreate the exact same model, including its weights and the optimizer(r2/3)
#new_model = models.load_model('/content/drive/MyDrive/Colab Notebooks/my_model.h5')
# Show the model architecture
#new_model.summary()

In [ ]:
import pandas as pd
import numpy as np
test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/average GC full dataset new for colab.csv')
Xtest=test.iloc[:,:5].values
Ytest = test.iloc[:,5:].values
print(Xtest[0])


In [ ]:
prediction=new_model.predict(Xtest)
print(prediction)
print(Ytest)

#import numpy as np

#Defining MAPE function
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

LR_MAPE= MAPE(Ytest,prediction)
print("MAPE: ",LR_MAPE)
#print(Ytest[0,2])

In [ ]:
#target=[(15,10,45)]
#weight=[(0.5,0.1,1)]

def error_sum(input):
   
   pred=new_model.predict(input)
   error=np.abs(pred-target)
   error_s=(3*error[:,0])+(1.5*error[:,1])+(3*error[:,2])+(1*error[:,3])+(3*error[:,4])


   return error_s

In [ ]:
def fitness(input):
    ans=error_sum(input)

    if ans.all()==0:
        return 99999
    else:
        return np.abs(1/ans)


In [ ]:
target=[(15,10,45,2,25)]
weight=[(3,1.5,3,1,3)]
import random
#generate solutions
input=[]
for s in range(20):
    NPCtype=random.randint(1,2)
    
    NPCweight=random.randrange(50000,61000,100)/1000
    loading=random.randrange(25,60,1)/100
    KOHconc=random.choice([1,3,0.75])
    #ratio=random.randrange(7,11,1)
    current=random.randrange(100,1101,100)

    input.append([(NPCtype,NPCweight,loading,KOHconc,current)])

input=np.asarray(input)
mating_pool=input.reshape(input.shape[0],input.shape[2])
print(mating_pool)


In [ ]:
fitness_mating_pool=fitness(mating_pool)
mix_mating_fitness=np.column_stack((mating_pool,fitness_mating_pool))
#print(mix_mating_fitness)

mixsort=mix_mating_fitness[mix_mating_fitness[:, 5].argsort()]

print(mixsort)
parents_pool=mixsort[:,0:5]
parents_results=mixsort[:,5]

print(parents_pool)
print(parents_results)


In [ ]:
#select first four as parents(best 4)
def parent_selection(mixsort):
    fitness_mating_pool=fitness(mating_pool)
    mix_mating_fitness=np.column_stack((mating_pool,fitness_mating_pool))

    mixsort=mix_mating_fitness[mix_mating_fitness[:, 5].argsort()]
    mixsort =mixsort[::-1]
    parents_pool=mixsort[:,0:5]
    parents_results=mixsort[:,5]
    return parents_pool[0:5]

In [ ]:
print(parent_selection(mating_pool))

In [ ]:
#select first four as parents(best 4)
#for i in range(4):
 #   print(mating_pool[i],fitness_initial_pool[i])

In [ ]:
#select mating parents(2)

#def parent_selection(mating_pool):
 #   fitness_initial_pool=fitness(mating_pool)
  #  fitness_initial_pool.sort()
   # fitness_initial_pool =fitness_initial_pool[::-1]
    #return mating_pool[0:4]
    
        
#test=parent_selection(mating_pool)
#print(test)

In [ ]:
# crossover(for mating)
def uniform_crossover(A,B,C,D,P):
    for i in range(len(P)):
        if P[i]<0.5:
          temp=A[i]
          A[i]=B[i]
          B[i]=C[i]
          C[i]=D[i]
          D[i]=temp
    return A,B,C,D

def crossover_process(mating_pool):
    A=mating_pool[0]
    B=mating_pool[1]
    C=mating_pool[2]
    D=mating_pool[3]
    P=np.random.rand(4)
    crossover_pool=uniform_crossover(A,B,C,D,P)
    crossover_pool=np.asarray(crossover_pool)
    return crossover_pool


crossover_pool=crossover_process(parent_selection(mating_pool))
print(crossover_pool)



In [ ]:
#mutation!
def mutation(crossover_pool):
    mutation_rate=random.uniform(0.8,1.2)
    mutation_pool=crossover_pool*mutation_rate
    return mutation_pool

mutation_pool=mutation(crossover_pool)
print(mutation_pool)
#print(mutation_pool)
#mutation_rate=random.uniform(0.98,1.02)
#mutation_pool=crossover_pool*mutation_rate


In [ ]:
#survivor selection(1)
#print(mating_pool)
#new_mating_pool=np.insert(mutation_pool,0,mating_pool,axis=0)
#print(new_mating_pool)

In [ ]:
#survival selection(2)
#fitness_new_pool=fitness(new_mating_pool)
#print(fitness_new_pool)

#fitness_new_pool.sort()
#fitness_new_pool =fitness_new_pool[::-1]
#print(fitness_new_pool)
#fitness_new_pool=fitness_new_pool[:20]
#print(fitness_new_pool)
#for i in range(20):
#    print(new_mating_pool[i],fitness_new_pool[i])

In [ ]:
#survival selection by def
def survivor_selection():
    new_mating_pool=np.insert(mutation(crossover_process(parent_selection(mating_pool))),0,mating_pool,axis=0)
    fitness_new_pool=fitness(new_mating_pool)
    mix_new_fitness=np.column_stack((new_mating_pool,fitness_new_pool))

    mixsortnew=mix_new_fitness[mix_new_fitness[:, 4].argsort()]
    mixsortnew =mixsortnew[::-1]
    mixsortnew=mixsortnew[:]
    new_mating_pool=mixsortnew[0:20,0:5]
    return new_mating_pool


test2=survivor_selection()
print(test2)

In [ ]:
def genetic_algorithm():
    parents=parent_selection(mating_pool)
    
    
    cross_p=crossover_process(parents)
   
    mutant_p=mutation(cross_p)
    survived_p=survivor_selection()
    return survived_p
    


print(genetic_algorithm())


In [ ]:
initial_gen=0
max_gen=1000



while initial_gen<max_gen:
   x_new=genetic_algorithm()
   initial_gen+=1 
   mating_pool=x_new
   print("gen",initial_gen,"=",x_new)
   test=new_model.predict(initial_gen)
   print("best results:",test[0])
   fittest=MAPE(test[0],target)
   print("Mape of Fittest:",fittest)
   if fittest==20 : break
   

In [ ]:
print(test[0],mating_pool[0])
print(new_model.predict(mating_pool))
